# **Classification of mails as spam or not-spam**


**Installing nessesary packages** 


In [1]:
!pip install texthero
!pip install catboost

**Importing libraries**

In [2]:
import texthero as hero
import numpy as np
import pandas as pd


**Reading the mail dataset**

In [3]:
dataset=pd.read_csv('emails.csv');

**Cleaning the messages**

In [4]:
from texthero import stopwords
stop_words=stopwords.DEFAULT
stop_words=stop_words.union(set(["subject"]))
corpus=[]
dataset['text']=hero.clean(dataset['text'])
dataset['text']=hero.remove_stopwords(dataset['text'],stop_words)
dataset['text']=hero.preprocessing.stem(dataset['text'],stem='porter',language='english')
l=dataset.shape
for i in range(0,l[0]):
  corpus.append(dataset['text'][i])

**Making of sparse matrix**

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=25000)
x=cv.fit_transform(corpus).toarray()
y=dataset['spam']

**Splliting into test and training data**

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 0)

**Scalling the data in sparse matrix**

In [7]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

**Appling Principal-component-analysis to reduce the dimentiality of sparse matrix**

In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 1500)
x_train = pca.fit_transform(x_train)
x_test = pca.transform(x_test)

#**Classification using catboost model**

**Initializing and training CatBoostClassifier object**

In [9]:
from catboost import CatBoostClassifier
classifier = CatBoostClassifier()
classifier.fit(x_train, y_train)

Learning rate set to 0.019734
0:	learn: 0.6627694	total: 659ms	remaining: 10m 57s
1:	learn: 0.6360086	total: 1.05s	remaining: 8m 42s
2:	learn: 0.6131718	total: 1.44s	remaining: 7m 59s
3:	learn: 0.5851083	total: 1.85s	remaining: 7m 40s
4:	learn: 0.5617706	total: 2.25s	remaining: 7m 28s
5:	learn: 0.5406093	total: 2.66s	remaining: 7m 20s
6:	learn: 0.5193080	total: 3.05s	remaining: 7m 12s
7:	learn: 0.4975538	total: 3.45s	remaining: 7m 7s
8:	learn: 0.4781455	total: 3.84s	remaining: 7m 2s
9:	learn: 0.4616278	total: 4.23s	remaining: 6m 59s
10:	learn: 0.4445855	total: 4.64s	remaining: 6m 57s
11:	learn: 0.4276106	total: 5.04s	remaining: 6m 55s
12:	learn: 0.4127923	total: 5.46s	remaining: 6m 54s
13:	learn: 0.3992168	total: 5.85s	remaining: 6m 51s
14:	learn: 0.3896278	total: 6.26s	remaining: 6m 51s
15:	learn: 0.3806932	total: 6.66s	remaining: 6m 49s
16:	learn: 0.3701041	total: 7.06s	remaining: 6m 48s
17:	learn: 0.3609458	total: 7.47s	remaining: 6m 47s
18:	learn: 0.3515627	total: 7.88s	remaining: 

**Computing the accuracy**

In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[878   7]
 [ 25 236]]


0.9720767888307156

#**Classification using ANN model**


In [11]:
import tensorflow as tf

**Initializing and training ANN object**

In [12]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=700, activation='relu'))
ann.add(tf.keras.layers.Dense(units=700, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
144/144 [==============================] - 2s 12ms/step - loss: 0.5047 - accuracy: 0.8900
Epoch 2/100
144/144 [==============================] - 2s 13ms/step - loss: 0.0665 - accuracy: 0.9795
Epoch 3/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0173 - accuracy: 0.9950
Epoch 4/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0086 - accuracy: 0.9980
Epoch 5/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0077 - accuracy: 0.9983
Epoch 6/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0031 - accuracy: 0.9993
Epoch 7/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0048 - accuracy: 0.9991
Epoch 8/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0032 - accuracy: 0.9991
Epoch 9/100
144/144 [==============================] - 2s 12ms/step - loss: 0.0031 - accuracy: 0.9993
Epoch 10/100
144/144 [==============================] - 2s 12ms/step - loss: 0.001

**Computing the accuracy**

In [14]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[871  14]
 [ 16 245]]


0.9738219895287958